In [1]:
#
# Copyright © 2019 Sunho Kim. All rights reserved.
#

In [2]:
cd ..

/gorani/gorani/backend/dataserver


In [3]:
from pyspark.sql import SparkSession, DataFrame

spark = SparkSession\
    .builder\
    .appName('App Name')\
    .getOrCreate()
sc = spark.sparkContext

In [4]:
# parameters
book_number = 5

In [5]:
from gorani.spark import read_api_all, read_data_all

books_df = read_data_all(spark, 'books').select('id')

eb_df = read_data_all(spark, 'experienced_books')
eb_df.show()

+-------+-------+
|user_id|book_id|
+-------+-------+
|     10|      4|
+-------+-------+



In [6]:
import pyspark.sql.functions as F
popular_book_df = eb_df.groupBy('book_id').count().orderBy(F.desc('count')).limit(book_number)
popular_book_df.show()
popular_book_df = popular_book_df.select(F.col('book_id').alias('id'))

+-------+-----+
|book_id|count|
+-------+-----+
|      4|    1|
+-------+-----+



In [7]:
from gorani.spark import write_api

result_df = popular_book_df
if popular_book_df.count() < book_number:
    needed_number = book_number - popular_book_df.count()
    random_book_df = books_df.join(popular_book_df, books_df['id'] == popular_book_df['id'], 'left_anti')\
        .select(F.col('id'))\
        .orderBy(F.rand()).limit(needed_number)
    result_df = popular_book_df.union(random_book_df)

write_api('start_recommended_books', result_df)
result_df.show()

Py4JJavaError: An error occurred while calling o90.jdbc.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 18.0 failed 4 times, most recent failure: Lost task 1.3 in stage 18.0 (TID 1056, 183.99.225.238, executor 0): java.sql.BatchUpdateException: Batch entry 0 INSERT INTO start_recommended_books ("id") VALUES (2) was aborted: ERROR: duplicate key value violates unique constraint "start_recommended_books_pkey"
  Detail: Key (id)=(2) already exists.  Call getNextException to see other errors in the batch.
	at org.postgresql.jdbc.BatchResultHandler.handleError(BatchResultHandler.java:148)
	at org.postgresql.core.v3.QueryExecutorImpl.processResults(QueryExecutorImpl.java:2184)
	at org.postgresql.core.v3.QueryExecutorImpl.execute(QueryExecutorImpl.java:481)
	at org.postgresql.jdbc.PgStatement.executeBatch(PgStatement.java:840)
	at org.postgresql.jdbc.PgPreparedStatement.executeBatch(PgPreparedStatement.java:1538)
	at org.apache.spark.sql.execution.datasou